<a href="https://colab.research.google.com/github/faizanali02/googlecolab/blob/main/233574_ai_a08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install mlrose-hiive==2.2.4 joblib==1.2.0 --quiet

import pandas as pd
import os, json, random
import numpy as np
import mlrose_hiive as mlrose
from collections import deque

# PART A — DATA SETUP & MANAGEMENT

def log_activity(message):
    """Appends each performed action into delivery_log.txt."""
    with open("delivery_log.txt", "a") as log:
        log.write(message + "\n")
    print(message)

def load_delivery_data(filename):
    """Loads data from CSV using pandas. Handle missing files using try-except."""
    try:
        df = pd.read_csv(filename)
        data = df.to_dict(orient='records')
        avg_distance = df['Distance(km)'].mean()
        log_activity(f"Data Loaded Successfully\nTotal Deliveries: {len(df)} | Average Distance: {avg_distance:.2f} km")
        return data, df
    except FileNotFoundError:
        log_activity("Error: File not found. Please upload delivery_data.csv.")
        return [], pd.DataFrame()

def search_order(order_id, data):
    """Searches and returns a dictionary for the given OrderID."""
    for record in data:
        if record.get("OrderID") == order_id:
            log_activity(f"OrderID: {order_id} found → {record}")
            return record
    log_activity(f"OrderID: {order_id} not found.")
    return None

# PART B — ROUTE SEARCH & GENETIC OPTIMIZATION

def bfs_route(graph, start, goal):
    """Implement BFS to find the shortest route between zones."""
    visited = set()
    queue = deque([(start, [start])])
    while queue:
        node, path = queue.popleft()
        if node == goal:
            log_activity(f"BFS Shortest Route: {' → '.join(path)}")
            return path
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
    return None

def greedy_route_selection(data):
    """Apply Greedy Algorithm using heuristic h(n)=Distance(km) to find the minimum delivery route."""
    if len(data) == 0:
        log_activity("No data for greedy route selection.")
        return [], 0

    unvisited = data.copy()
    route = []
    total_distance = 0.0
    current = unvisited.pop(0)
    route.append(current['OrderID'])

    while unvisited:
        next_delivery = min(unvisited, key=lambda x: x['Distance(km)'])
        total_distance += next_delivery['Distance(km)']
        route.append(next_delivery['OrderID'])
        unvisited.remove(next_delivery)

    log_activity(f"Greedy Algorithm Results: Route: {route}, Total Distance = {total_distance:.2f} km")
    return route, total_distance

def genetic_route_optimization(data):
    """Apply Genetic Algorithm using mlrose to minimize heuristic h(n)=Total Distance(km)."""
    if len(data) == 0:
        log_activity("No data for genetic optimization.")
        return [], 0

    # Create a pseudo-distance matrix using distance differences
    n = len(data)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = abs(data[i]['Distance(km)'] - data[j]['Distance(km)'])
            distances[i, j] = d
            distances[j, i] = d

    dist_list = [(i, j, distances[i, j]) for i in range(n) for j in range(i+1, n)]
    fitness = mlrose.TravellingSales(distances=dist_list)
    problem = mlrose.TSPOpt(length=n, fitness_fn=fitness, maximize=False)

    best_state, best_fitness, _ = mlrose.genetic_alg(problem, random_state=2, mutation_prob=0.2, max_iters=200)
    total_distance = -best_fitness if best_fitness < 0 else best_fitness

    log_activity(f"[Genetic Algorithm Results]\nOptimal Route Found: {best_state}\nTotal Distance (Genetic) = {total_distance:.2f} km\nGenerations: 200 | Best Fitness: {best_fitness:.2f}")
    return best_state, total_distance

# PART C — INTELLIGENT AGENT SYSTEM

def reflex_agent(order):
    """If TrafficDelay > 10, reroute delivery; otherwise, proceed."""
    if order['TrafficDelay(min)'] > 10:
        print(f"[Reflex Agent] 🚦 High delay ({order['TrafficDelay(min)']} mins) — Rerouting delivery OrderID {order['OrderID']}")
    else:
        print(f"[Reflex Agent] Normal route — Proceeding with delivery OrderID {order['OrderID']}")

def goal_based_agent(order):
    """Print plan of actions for order fulfillment."""
    plan = "Pickup → Drive → Deliver → Confirm"
    print(f"[Goal-Based Agent] Plan for OrderID {order['OrderID']}: {plan}")

def utility_based_agent(order):
    """Return computed utility value for delivery decision."""
    utility = (1 / order['Distance(km)']) * (20 - order['TrafficDelay(min)'])
    print(f"[Utility Agent] OrderID {order['OrderID']} Utility Score: {utility:.2f}")
    return utility

def print_peas_model():
    """Display PEAS model for Reflex, Goal-based, and Utility-based agents."""
    peas = {
        "Performance": "Minimize delay, timely delivery",
        "Environment": "City grid with real-time traffic",
        "Actuators": "Vehicle control systems",
        "Sensors": "GPS, traffic feed"
    }
    print("PEAS Model:")
    for k, v in peas.items():
        print(f"{k}: {v}")

# MAIN EXECUTION

if __name__ == "__main__":
    # Load data
    data, df = load_delivery_data("delivery_data.csv")

    if len(data) > 0:
        # Search specific order
        search_order(105, data)

        # Example graph for BFS
        graph = {
            'A': ['B', 'C'],
            'B': ['A', 'D', 'E'],
            'C': ['A', 'F'],
            'D': ['B'],
            'E': ['B', 'F'],
            'F': ['C', 'E']
        }
        bfs_route(graph, 'A', 'F')

        # Greedy Algorithm
        greedy_route_selection(data)

        # Genetic Algorithm
        genetic_route_optimization(data)

        # Agents demonstration (first order)
        order = data[0]
        reflex_agent(order)
        goal_based_agent(order)
        utility_based_agent(order)
        print_peas_model()


Data Loaded Successfully
Total Deliveries: 20 | Average Distance: 7.03 km
OrderID: 105 found → {'OrderID': 105, 'Destination': 'Zone E', 'Distance(km)': 10.68, 'TrafficDelay(min)': 20}
BFS Shortest Route: A → C → F
Greedy Algorithm Results: Route: [101, 109, 117, 114, 119, 106, 111, 102, 116, 108, 104, 113, 120, 110, 103, 112, 118, 105, 107, 115], Total Distance = 137.28 km
[Genetic Algorithm Results]
Optimal Route Found: [ 6  4 11  2  9 19 12  3 15 10  5 13  0 16  8 18  1  7 17 14]
Total Distance (Genetic) = 18.44 km
Generations: 200 | Best Fitness: 18.44
[Reflex Agent] Normal route — Proceeding with delivery OrderID 101
[Goal-Based Agent] Plan for OrderID 101: Pickup → Drive → Deliver → Confirm
[Utility Agent] OrderID 101 Utility Score: 3.51
PEAS Model:
Performance: Minimize delay, timely delivery
Environment: City grid with real-time traffic
Actuators: Vehicle control systems
Sensors: GPS, traffic feed
